<a href="https://colab.research.google.com/github/talchil/Deadlock-detection/blob/main/bdd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install pyeda

In [14]:
from pyeda.boolalg.bdd import bddvar
from pyeda.boolalg.expr import Or

In [15]:
# ===========================
# CẤU HÌNH 1-PETRI SAFE NET
# ===========================

# --------- Nhập số place ---------
m = int(input("Nhập số place m: "))
P = [bddvar(f"p{i}") for i in range(m)]

# ---------------------------------------
# R: reachable markings dạng ONE-HOT
# Cho người dùng nhập vào
# ---------------------------------------
R = []
num_R = int(input("Nhập số lượng reachable markings: "))

print("\n--- Nhập từng marking (one-hot) ---")
for k in range(num_R):
    print(f"Marking {k} (nhập {m} số 0/1 cách nhau bởi khoảng trắng):")
    row = list(map(int, input().split()))
    if len(row) != m:
        raise ValueError("Số phần tử không khớp với m")
    if sum(row) != 1:
        raise ValueError("Marking phải dạng ONE-HOT (tổng = 1)")
    R.append(row)

# ---------------------------------------
# F: firing conditions dạng ONE-HOT
# Cho người dùng nhập vào
# ---------------------------------------
F = []
num_F = int(input("\nNhập số lượng firing transitions: "))

print("\n--- Nhập từng firing transition (điều kiện để transition xảy ra)")
for k in range(num_F):
    print(f"Transition {k} (nhập {m} số 0/1):")
    row = list(map(int, input().split()))
    if len(row) != m:
        raise ValueError("Số phần tử không khớp với m")
    if sum(row) != 1:
        raise ValueError("Firing phải dạng ONE-HOT (tổng = 1)")
    F.append(row)

Nhập số place m: 3
Nhập số lượng reachable markings: 3

--- Nhập từng marking (one-hot) ---
Marking 0 (nhập 3 số 0/1 cách nhau bởi khoảng trắng):
1 0 0
Marking 1 (nhập 3 số 0/1 cách nhau bởi khoảng trắng):
0 1 0
Marking 2 (nhập 3 số 0/1 cách nhau bởi khoảng trắng):
0 0 1

Nhập số lượng firing transitions: 2

--- Nhập từng firing transition (điều kiện để transition xảy ra)
Transition 0 (nhập 3 số 0/1):
1 0 0
Transition 1 (nhập 3 số 0/1):
0 1 0


In [16]:
# ===========================
# TẠO BDD Firing-Enabled từ F
# ===========================

def marking_to_bdd(marking, place_vars):
    """Chuyển một marking 0/1 thành BDD"""
    term = None
    for i, val in enumerate(marking):
        literal = place_vars[i] if val==1 else ~place_vars[i]
        if term is None:
            term = literal
        else:
            term = term & literal
    return term

# Chuyển từng marking trong F thành BDD và nối bằng OR
firing_bdd = None
for mark in F:
    bdd_term = marking_to_bdd(mark, P)
    if firing_bdd is None:
        firing_bdd = bdd_term
    else:
        firing_bdd = firing_bdd | bdd_term

In [17]:
# ===========================
# KIỂM TRA DEADLOCK
# ===========================

def check_deadlock_safe(R, place_vars, firing_bdd):
    not_firing_bdd = ~firing_bdd

    for idx, marking in enumerate(R):
        assignment = {place_vars[i]: marking[i] for i in range(len(place_vars))}
        res = not_firing_bdd.restrict(assignment)
        if res.is_one():
            print("Deadlock detected at marking:", marking)
            return

    print("No deadlock found")

In [18]:
# ===========================
# CHẠY HÀM
# ===========================

check_deadlock_safe(R, P, firing_bdd)

Deadlock detected at marking: [0, 0, 1]
